# A re-mapping of reseller_country to fallback table values

In [1]:
options(repr.matrix.max.cols=50, repr.matrix.max.rows=100)

In [2]:
# install.packages("pkgcond")

In [3]:
library(dplyr)
library(data.table)
library(partykit)
library(tictoc)
library(caret)
library(e1071)
library(randomForest)
library(ranger)

#for 3d plotting
library(akima)
library(plotly)

# for prep data
library(rPython)
library(stringr)
library(pbapply)
library(stringdist)
library(data.table)
library(dominanceanalysis)


getwd()


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


Loading required package: grid

Loading required package: libcoin

Loading required package: mvtnorm

Loading required package: lattice

Loading required package: ggplot2

randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.


Attaching package: ‘randomForest’


The following object is masked from ‘package:ggplot2’:

    margin


The following object is masked from ‘package:dplyr’:

    combine



Attaching package: ‘ranger’


The following object is masked from ‘package:randomForest’:

    importance



Attaching package: ‘plotly’


The following object is masked from ‘package:ggplot2’:

    last_plot


The following object is masked from ‘package:sta

[1] "/home/jupyter/local/Domains_202003/scripts/phaseII_05_metalearning"

In [4]:

source('../orig/functions.R')

source('../orig/functions_models.R')

source('../phaseII_03_forest/functions_eval.R')


source('../phaseII_03_forest/load_prep_data_expiry.R')

# expiry_train_prepped_2_1 (list, less df's w/ 0 obs)
# expiry_test_prepped_2_1  (list, less df's w/ 0 obs)
# expiry_train_df_1 (above, rbound)
# expiry_test_df_1 (above, rbound)
# expiry_train_df_sub (subset vars)                                        
# expiry_test_df_sub (subset vars)

## create country -> geo lookup table
### incorrect location, abandoned

In [5]:
system("gsutil cp gs://data_input/npv_fallback_tables /home/jupyter/local/Domains_202003/data/npv_fallback_tables")
npv_fallback_tables <- load("/home/jupyter/local/Domains_202003/data/npv_fallback_tables")


In [6]:
npv_fallback_tables

[1] "npv_fallback_first_final"        "npv_fallback_first_geo_arpt"    
 [3] "npv_fallback_first_geo_arpt_tld" "npv_fallback_first_tld_arpt"    
 [5] "npv_fallback_second_final"       "npv_fallback_second_geo"        
 [7] "npv_fallback_second_tld_geo"     "npv_fallback_third_final"       
 [9] "npv_fallback_third_geo"          "npv_fallback_third_tld_geo"

In [13]:
head(npv_fallback_third_tld_geo)

tld,reseller_geo,expiring_domains,renewed_domains,renewal_rate,index
<chr>,<chr>,<int>,<int>,<dbl>,<chr>
host,Canada,215,178,0.828,hostCanada
host,China,243,152,0.626,hostChina
host,EU,652,595,0.913,hostEU
host,India,237,189,0.797,hostIndia
host,United States,2155,1663,0.772,hostUnited States
in.net,EU,183,140,0.765,in.netEU


### uploading tab from https://docs.google.com/spreadsheets/d/1ybBC_KLGKYG3D54px3e2k0LP0LF5FjJ2uMAXTAUnzuU/edit#gid=427645846

In [20]:
# system("gsutil cp gs://data_input/PredictiveModelAnalysis_AnomalyRegistrars.csv /home/jupyter/local/Domains_202003/data/PredictiveModelAnalysis_AnomalyRegistrars.csv")
# anomalyDF <- read.csv("/home/jupyter/local/Domains_202003/data/PredictiveModelAnalysis_AnomalyRegistrars.csv")

anomalyDF$registrar <- trimws(anomalyDF$registrar, which = c("both"))
anomalyDF$reseller <- trimws(anomalyDF$reseller, which = c("both"))
anomalyDF$reseller_country <- trimws(anomalyDF$reseller_country, which = c("both"))
anomalyDF$reseller_geo <- trimws(anomalyDF$reseller_geo, which = c("both"))

anomalyDF <- apply(anomalyDF, 2, function(x) trimws(x,which = c("both")) )

In [21]:
head(anomalyDF)

registrar,reseller,reseller_country,reseller_geo,notes
007names,007names,United States,United States,
0101 internet,0101 internet,Hong Kong,China,
1&1 internet,1&1 internet,Germany,EU,
101 domain,101 domain,United States,United States,
101 domain,101 domain,United States,United States,
101domain discovery,101domain discovery,Germany,EU,


In [27]:
geoLookupDF <- unique(anomalyDF[,c('reseller','reseller_country','reseller_geo')])

In [28]:
geoLookupDF

reseller,reseller_country,reseller_geo
007names,United States,United States
0101 internet,Hong Kong,China
1&1 internet,Germany,EU
101 domain,United States,United States
101domain discovery,Germany,EU
10dencehispahard,Spain,EU
10dollar,Canada,Canada
123-reg,United Kingdom,EU
123cheapdomains.com,Others,Others
123connect,United Kingdom,EU


In [29]:
dim(geoLookupDF)

[1] 1481    3

In [30]:
write.csv(geoLookupDF, "../../data/output/geoLookup.csv", row.names=FALSE)

## append reseller_geo to train & test data

In [32]:
# names as used in _list.R script
train_list = expiry_train_prepped_2_1
test_list = expiry_test_prepped_2_1

In [33]:
length(train_list)

[1] 1723

In [34]:
length(test_list)

[1] 1405

In [50]:
for (i in 1:length(train_list)){
    cat(length(unique(train_list[[i]]$tld_registrar_index)))}

1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111

In [58]:
for (i in 1:length(train_list)){
    if (names(train_list)[[i]]!=unique(train_list[[i]]$tld_registrar_index)){
        cat(i)}}

In [57]:
names(train_list)[[1722]]
unique(train_list[[1722]]$tld_registrar_index)
head(train_list[[1722]])

[1] "websitezhengzhou century connect"

[1] "websitezhengzhou century connect"

renewal_type,renewed_count,expiry_date,domain_id,domain,creation_date,status,tld,registrar,reseller,reseller_country,region,reg_period,registrant_country,renewal_status,renew_mbg,renew_type,autorenew_type,renew_date,renew_registrar,renew_reseller,reg_revenue,reg_arpt,renew_period,renew_domain_revenue,renew_arpt,reg_arpt_org,tld_registrar_index,sld,sld_type,sld_length,sld_type2,day_domains,log_reg_arpt,gibb_score,pattern,cluster,pattern_score,pattern_domain_count
<chr>,<int>,<date>,<int>,<chr>,<date>,<chr>,<fct>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<fct>,<int>,<chr>,<chr>,<date>,<chr>,<chr>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<fct>,<int>,<chr>,<int>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<int>
FirstTime,1,2020-01-09,90424822,woyaozhizhu.website,2019-01-09,Active,website,zhengzhou century connect,zhengzhou century connect,China,China,1,NA,Not Renewd,0,auto-renewal,unrealized,2020-01-09,Zhengzhou Century Connect,Zhengzhou Century Connect,0.75,0.75,1,5,5,0.75,websitezhengzhou century connect,woyaozhizhu,l,11,11l,1,-0.2876821,0.99,woyaozhizhu,1,1.00000000,1
FirstTime,1,2020-01-26,91955393,sunbet26.website,2019-01-26,Active,website,zhengzhou century connect,zhengzhou century connect,China,China,1,NA,Not Renewd,0,auto-renewal,unrealized,2020-01-26,Zhengzhou Century Connect,Zhengzhou Century Connect,0.75,0.75,1,5,5,0.75,websitezhengzhou century connect,sunbet26,ln,8,8ln,39,-0.2876821,3.35,1,sunbet,0.02564103,39
FirstTime,1,2020-01-26,91955268,sunbet1.website,2019-01-26,Active,website,zhengzhou century connect,zhengzhou century connect,China,China,1,NA,Not Renewd,0,auto-renewal,unrealized,2020-01-26,Zhengzhou Century Connect,Zhengzhou Century Connect,0.75,0.75,1,5,5,0.75,websitezhengzhou century connect,sunbet1,ln,7,7ln,39,-0.2876821,3.35,1,sunbet,0.02564103,39
FirstTime,1,2020-01-26,91955338,sunbet15.website,2019-01-26,Active,website,zhengzhou century connect,zhengzhou century connect,China,China,1,NA,Not Renewd,0,auto-renewal,unrealized,2020-01-26,Zhengzhou Century Connect,Zhengzhou Century Connect,0.75,0.75,1,5,5,0.75,websitezhengzhou century connect,sunbet15,ln,8,8ln,39,-0.2876821,3.35,1,sunbet,0.02564103,39
FirstTime,1,2020-01-26,91955293,sunbet6.website,2019-01-26,Active,website,zhengzhou century connect,zhengzhou century connect,China,China,1,NA,Not Renewd,0,auto-renewal,unrealized,2020-01-26,Zhengzhou Century Connect,Zhengzhou Century Connect,0.75,0.75,1,5,5,0.75,websitezhengzhou century connect,sunbet6,ln,7,7ln,39,-0.2876821,3.35,1,sunbet,0.02564103,39
FirstTime,1,2020-01-26,91955478,sunbet42.website,2019-01-26,Active,website,zhengzhou century connect,zhengzhou century connect,China,China,1,NA,Not Renewd,0,auto-renewal,unrealized,2020-01-26,Zhengzhou Century Connect,Zhengzhou Century Connect,0.75,0.75,1,5,5,0.75,websitezhengzhou century connect,sunbet42,ln,8,8ln,39,-0.2876821,3.35,1,sunbet,0.02564103,39


In [53]:
i=1
train_temp <- merge(train_list[[i]],geoLookupDF,on=c('reseller','reseller_country'), how='left')
head(train_temp)

reseller,reseller_country,renewal_type,renewed_count,expiry_date,domain_id,domain,creation_date,status,tld,registrar,region,reg_period,registrant_country,renewal_status,renew_mbg,renew_type,autorenew_type,renew_date,renew_registrar,renew_reseller,reg_revenue,reg_arpt,renew_period,renew_domain_revenue,renew_arpt,reg_arpt_org,tld_registrar_index,sld,sld_type,sld_length,sld_type2,day_domains,log_reg_arpt,gibb_score,pattern,cluster,pattern_score,pattern_domain_count,reseller_geo
<chr>,<chr>,<chr>,<int>,<date>,<int>,<chr>,<date>,<chr>,<fct>,<chr>,<chr>,<int>,<chr>,<fct>,<int>,<chr>,<chr>,<date>,<chr>,<chr>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<fct>,<int>,<chr>,<int>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<int>,<chr>
1&1 internet,Germany,FirstTime,1,2020-01-21,91584433,racefor.fun,2019-01-21,Active,fun,1&1 internet,Non China,1,NA,Renewed,0,auto-renewal,realized,2020-01-21,1&1 Internet,1&1 Internet,15,15,1,15,15,15,fun1&1 internet,racefor,l,7,7l,1,2.70805,6.77,racefor,1,1,1,EU
1&1 internet,Germany,FirstTime,1,2020-01-30,92254793,united4.fun,2019-01-30,Active,fun,1&1 internet,Non China,1,NA,Renewed,0,auto-renewal,realized,2020-01-30,1&1 Internet,1&1 Internet,15,15,1,15,15,15,fun1&1 internet,united4,ln,7,7ln,1,2.70805,8.65,united4,1,1,1,EU
1&1 internet,Germany,FirstTime,1,2020-02-04,92827233,bereal.fun,2019-02-04,Active,fun,1&1 internet,Non China,1,NA,Renewed,0,auto-renewal,realized,2020-02-04,1&1 Internet,1&1 Internet,15,15,1,15,15,15,fun1&1 internet,bereal,l,6,6l,1,2.70805,13.09,bereal,1,1,1,EU
1&1 internet,Germany,FirstTime,1,2020-02-12,93490823,dogsoutdoors.fun,2019-02-12,Active,fun,1&1 internet,Non China,1,NA,Not Renewd,0,auto-renewal,unrealized,2020-02-12,1&1 Internet,1&1 Internet,15,15,1,15,15,15,fun1&1 internet,dogsoutdoors,l,12,12l,1,2.70805,2.44,dogsoutdoors,1,1,1,EU
1&1 internet,Germany,FirstTime,1,2020-02-15,93767978,rosalux.fun,2019-02-15,Active,fun,1&1 internet,Non China,1,NA,Renewed,0,auto-renewal,realized,2020-02-15,1&1 Internet,1&1 Internet,15,15,1,15,15,15,fun1&1 internet,rosalux,l,7,7l,1,2.70805,2.34,rosalux,1,1,1,EU
1&1 internet,Germany,FirstTime,1,2020-02-16,93823708,verygood.fun,2019-02-16,Active,fun,1&1 internet,Non China,1,NA,Renewed,0,auto-renewal,realized,2020-02-16,1&1 Internet,1&1 Internet,15,15,1,15,15,15,fun1&1 internet,verygood,l,8,8l,1,2.70805,3.64,verygood,1,1,1,EU


In [59]:
train_list_2 <- lapply(train_list, function(x){
 x <- merge(x,geoLookupDF,on=c('reseller','reseller_country'), how='left')
 return(x)
})


In [60]:
length(train_list_2)
dim(train_list_2[[1]])
head(train_list_2[[1]])

[1] 1723

[1] 19 40

reseller,reseller_country,renewal_type,renewed_count,expiry_date,domain_id,domain,creation_date,status,tld,registrar,region,reg_period,registrant_country,renewal_status,renew_mbg,renew_type,autorenew_type,renew_date,renew_registrar,renew_reseller,reg_revenue,reg_arpt,renew_period,renew_domain_revenue,renew_arpt,reg_arpt_org,tld_registrar_index,sld,sld_type,sld_length,sld_type2,day_domains,log_reg_arpt,gibb_score,pattern,cluster,pattern_score,pattern_domain_count,reseller_geo
<chr>,<chr>,<chr>,<int>,<date>,<int>,<chr>,<date>,<chr>,<fct>,<chr>,<chr>,<int>,<chr>,<fct>,<int>,<chr>,<chr>,<date>,<chr>,<chr>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<fct>,<int>,<chr>,<int>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<int>,<chr>
1&1 internet,Germany,FirstTime,1,2020-01-21,91584433,racefor.fun,2019-01-21,Active,fun,1&1 internet,Non China,1,NA,Renewed,0,auto-renewal,realized,2020-01-21,1&1 Internet,1&1 Internet,15,15,1,15,15,15,fun1&1 internet,racefor,l,7,7l,1,2.70805,6.77,racefor,1,1,1,EU
1&1 internet,Germany,FirstTime,1,2020-01-30,92254793,united4.fun,2019-01-30,Active,fun,1&1 internet,Non China,1,NA,Renewed,0,auto-renewal,realized,2020-01-30,1&1 Internet,1&1 Internet,15,15,1,15,15,15,fun1&1 internet,united4,ln,7,7ln,1,2.70805,8.65,united4,1,1,1,EU
1&1 internet,Germany,FirstTime,1,2020-02-04,92827233,bereal.fun,2019-02-04,Active,fun,1&1 internet,Non China,1,NA,Renewed,0,auto-renewal,realized,2020-02-04,1&1 Internet,1&1 Internet,15,15,1,15,15,15,fun1&1 internet,bereal,l,6,6l,1,2.70805,13.09,bereal,1,1,1,EU
1&1 internet,Germany,FirstTime,1,2020-02-12,93490823,dogsoutdoors.fun,2019-02-12,Active,fun,1&1 internet,Non China,1,NA,Not Renewd,0,auto-renewal,unrealized,2020-02-12,1&1 Internet,1&1 Internet,15,15,1,15,15,15,fun1&1 internet,dogsoutdoors,l,12,12l,1,2.70805,2.44,dogsoutdoors,1,1,1,EU
1&1 internet,Germany,FirstTime,1,2020-02-15,93767978,rosalux.fun,2019-02-15,Active,fun,1&1 internet,Non China,1,NA,Renewed,0,auto-renewal,realized,2020-02-15,1&1 Internet,1&1 Internet,15,15,1,15,15,15,fun1&1 internet,rosalux,l,7,7l,1,2.70805,2.34,rosalux,1,1,1,EU
1&1 internet,Germany,FirstTime,1,2020-02-16,93823708,verygood.fun,2019-02-16,Active,fun,1&1 internet,Non China,1,NA,Renewed,0,auto-renewal,realized,2020-02-16,1&1 Internet,1&1 Internet,15,15,1,15,15,15,fun1&1 internet,verygood,l,8,8l,1,2.70805,3.64,verygood,1,1,1,EU


In [33]:
system("gsutil cp /home/jupyter/local/Domains_202003/data/output/* gs://data_outputt/output/")
system("mv /home/jupyter/local/Domains_202003/data/output/* /home/jupyter/local/Domains_202003/data/")